## Spark DataFrame Basic Operations

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Ops").getOrCreate()
df=spark.read.csv('DataSets/appl_stock.csv',header=True,inferSchema=True)

In [2]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [3]:
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows


In [4]:
df.head(5)[2]

Row(Date=datetime.date(2010, 1, 6), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)

In [5]:
df.head(5)[2].asDict()

{'Date': datetime.date(2010, 1, 6),
 'Open': 214.379993,
 'High': 215.23,
 'Low': 210.750004,
 'Close': 210.969995,
 'Volume': 138040000,
 'Adj Close': 27.333178000000004}

In [9]:
df.filter("Open<500").show(2)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 2 rows


In [7]:
df.filter("Open<500").select(['Open','Close']).count()

1361

In [11]:
df.filter("Close>500 AND Open<500").select(['Open','Close']).show(2)

+----------+------------------+
|      Open|             Close|
+----------+------------------+
|499.529991|502.60002099999997|
|491.500008|502.20999900000004|
+----------+------------------+
only showing top 2 rows


In [13]:
df.filter((df["Close"]<500) & ~(df["Open"]>200)).select(['Open','Close']).show()

+------------------+------------------+
|              Open|             Close|
+------------------+------------------+
|192.36999699999998|        194.729998|
|        195.909998|        195.859997|
|        195.169994|        199.229994|
|        196.730003|        192.050003|
|192.63000300000002|        195.460001|
|        195.690006|194.11999699999998|
|        196.419996|196.19000400000002|
|        195.889997|195.12000700000002|
|        194.880001|        198.669994|
|        198.109995|200.37999299999998|
|        199.999998|        197.059998|
|        198.229998|            200.66|
|        197.380005|        202.000004|
|         92.699997|         93.699997|
|         94.730003|             94.25|
|         94.129997|         93.860001|
|         94.040001|         92.290001|
|         92.199997|         91.279999|
|         91.510002|         92.199997|
|         92.309998| 92.08000200000001|
+------------------+------------------+
only showing top 20 rows


In [14]:
df.filter(df["Low"]==197.16).show()

+----------+------------------+----------+------+------+---------+---------+
|      Date|              Open|      High|   Low| Close|   Volume|Adj Close|
+----------+------------------+----------+------+------+---------+---------+
|2010-01-22|206.78000600000001|207.499996|197.16|197.75|220441900|25.620401|
+----------+------------------+----------+------+------+---------+---------+



In [15]:
result=df.filter(df["Low"]==197.16).collect()

In [17]:
result[0]

Row(Date=datetime.date(2010, 1, 22), Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)

In [18]:
row=result[0]

In [19]:
row.asDict()

{'Date': datetime.date(2010, 1, 22),
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [20]:
row.asDict()['Open']

206.78000600000001

## GroupBy and Aggregate Functions

In [22]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Aggs").getOrCreate()
df=spark.read.csv('DataSets/sales_info.csv',header=True,inferSchema=True)

In [24]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [25]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [26]:
df.groupBy("Company")

GroupedData[grouping expressions: [Company], value: [Company: string, Person: string ... 1 more field], type: GroupBy]

In [27]:
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [28]:
df.groupBy("Company").max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [29]:
df.groupBy("Company").min().show()

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   APPL|     130.0|
|   GOOG|     120.0|
|     FB|     350.0|
|   MSFT|     124.0|
+-------+----------+



In [31]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [34]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [35]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [36]:
group_data =df.groupBy("Company")
group_data.agg({'Sales':'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [37]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number
df.select(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [38]:
df.select(avg("Sales").alias("Average Sales")).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [39]:
df.select(stddev("Sales").alias("Std Dev of Sales")).show()

+------------------+
|  Std Dev of Sales|
+------------------+
|250.08742410799007|
+------------------+



In [40]:
from pyspark.sql.functions import format_number
df.select(format_number(avg("Sales"),2).alias("Average Sales")).show()

+-------------+
|Average Sales|
+-------------+
|       360.58|
+-------------+



In [41]:
sales_std=df.select(stddev("Sales").alias("Std Dev of Sales"))
sales_std.select(format_number("Std Dev of Sales",2).alias("Formatted Std Dev of Sales")).show()

+--------------------------+
|Formatted Std Dev of Sales|
+--------------------------+
|                    250.09|
+--------------------------+



In [42]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [43]:
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [48]:
df.orderBy(df["Sales"].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

